In [ ]:
# 🧠 EyePop.ai - Person Detection Workshop
# To run this notebook, you must provide your API credentials below

# ✅ Step 1: Install required libraries
!pip install -q eyepop-sdk pillow matplotlib python-dotenv

# ✅ Step 2: Provide your EyePop credentials

In [ ]:
# flake8: noqa
# This is a simple example of how to use the EyePop SDK 
# to detect text in an image
from eyepop import EyePopSdk

# For pretty printing
import json

# .env configuration
from dotenv import load_dotenv

# For displaying images
from PIL import Image
import matplotlib.pyplot as plt

load_dotenv()

POP_ID = os.getenv("EYEPOP_POP_ID")
SECRET_KEY = os.getenv("EYEPOP_SECRET_KEY")

#example_image_path = './images/person1.jpg'
#example_image_path = './images/person2.jpg'
example_image_path = './images/person4.jpg'

with EyePopSdk.workerEndpoint(
    pop_id=POP_ID,
    secret_key=SECRET_KEY
) as endpoint:

    result = endpoint.upload(example_image_path).predict()

    print(json.dumps(result, indent=4))

    filtered_objects = [obj for obj in result["objects"] if obj["confidence"] > 0.80]
    filtered_result = {**result, "objects": filtered_objects}

    with Image.open(example_image_path) as image:
        plt.imshow(image)

    plot = EyePopSdk.plot(plt.gca())
    #plot.prediction(filtered_result)

    # draw white box over entire image
    plt.gca().add_patch(
        plt.Rectangle((0, 0), image.width, image.height, color="white", alpha=1)
    )

    # check 
    for obj in filtered_result["objects"]:
        if obj["classLabel"] != "person":
            continue

        # draw a red box around the person, just a border
        plt.gca().add_patch(
            plt.Rectangle(
            (obj["x"], obj["y"]),
            obj["width"],
            obj["height"],
            edgecolor="red",
            facecolor="none",
            linewidth=2,
            )
        )

        for point in obj["keyPoints"][0]['points']:
            print(point)
            plt.plot(point["x"], point["y"], "bo")
            plt.text(point["x"], point["y"], point["classLabel"], fontsize=12, color="blue")
    
    plt.show()
